# Projet de Machine Learning 
## Gym Members Exercise Dataset
### EHRHART Elsa, SLISSE Paul, STAUB Guillaume, HOSSEINI Arman
(Notebook Python)

#### I.Pre-proccessing des données

On va ici sans entrer dans le détail faire la même chose que les parties I.1, I.2 et I.3 du notebook R afin de préparer nos données.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
path="./gym_members_exercise_tracking.csv"
gym=pd.read_csv(path,sep=",",header=0)
gym.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Fat_Percentage,Water_Intake (liters),Workout_Frequency (days/week),Experience_Level,BMI
0,56,Male,88.3,1.71,180,157,60,1.69,1313.0,Yoga,12.6,3.5,4,3,30.20
1,46,Female,74.9,1.53,179,151,66,1.30,883.0,HIIT,33.9,2.1,4,2,32.00
2,32,Female,68.1,1.66,167,122,54,1.11,677.0,Cardio,33.4,2.3,4,2,24.71
3,25,Male,53.2,1.70,190,164,56,0.59,532.0,Strength,28.8,2.1,3,1,18.41
4,38,Male,46.1,1.79,188,158,68,0.64,556.0,Strength,29.2,2.8,3,1,14.39


In [4]:
gym["Gender"]=pd.Categorical(gym["Gender"],ordered=False)
gym["Workout_Type"]=pd.Categorical(gym["Workout_Type"],ordered=False)
gym["Experience_Level"]=pd.Categorical(gym["Experience_Level"],ordered=True)
gym.dtypes

In [10]:
gym["LWeight"]=gym["Weight (kg)"].map(lambda x:np.log(x))
gym["LBMI"]=gym["BMI"].map(lambda x:np.log(x))
gym["LWater_Intake"]=gym["Water_Intake (liters)"].map(lambda x:np.log(x))
gym["Fat_Percentage_S"]=gym["Fat_Percentage"].map(lambda x:x**2/100)
gym["SHeight"]=gym["Height (m)"].map(lambda x:np.sqrt(x))
del gym["Weight (kg)"]
del gym["BMI"]
del gym["Water_Intake (liters)"]
del gym["Fat_Percentage"]
del gym["Height (m)"]
gym.head()

,Age,Gender,Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,Workout_Frequency (days/week),Experience_Level,LBMI,LWeight,LWater_Intake,Fat_Percentage_S,SHeight
0,56,Male,180,157,60,1.69,1313.0,Yoga,4,3,3.407842,4.480740,1.252763,1.5876,1.307670
1,46,Female,179,151,66,1.30,883.0,HIIT,4,2,3.465736,4.316154,0.741937,11.4921,1.236932
2,32,Female,167,122,54,1.11,677.0,Cardio,4,2,3.207208,4.220977,0.832909,11.1556,1.288410
3,25,Male,190,164,56,0.59,532.0,Strength,3,1,2.912894,3.974058,0.741937,8.2944,1.303840
4,38,Male,188,158,68,0.64,556.0,Strength,3,1,2.666534,3.830813,1.029619,8.5264,1.337909


In [14]:
print(gym.isnull().sum())

Age                              0
Gender                           0
Max_BPM                          0
Avg_BPM                          0
Resting_BPM                      0
Session_Duration (hours)         0
Calories_Burned                  0
Workout_Type                     0
Workout_Frequency (days/week)    0
Experience_Level                 0
LBMI                             0
LWeight                          0
LWater_Intake                    0
Fat_Percentage_S                 0
SHeight                          0
dtype: int64


Il n'y a aucune valeur manquante dans le jeu de données.